## Day 20
#### Part 1

In [1]:
import numpy as np

In [2]:
# Read in data
input_file = open('day20_input.txt', 'r')
data = input_file.read()[:-2]

In [3]:
# Data wrangling
data = data.split("\n\n")
data = [tile.split("\n") for tile in data]
data = {int(tile[0][5:9]): tile[1:] for tile in data}
data = {k: np.array([list(line) for line in v]) for k, v in data.items()}

In [4]:
def get_tile_flips(arr):
    return [arr, np.fliplr(arr), np.flipud(arr), np.fliplr(np.flipud(arr))]


def get_borders(arr):
    return np.array([arr[0, :], arr[:, 0], arr[len(arr)-1, :], arr[:, len(arr)-1]])


def get_flipped_borders(arr):
    all_borders = []
    for tile in get_tile_flips(arr):
        all_borders.extend(get_borders(tile))
        
    return(np.array(all_borders))

In [5]:
borders = {k: get_flipped_borders(v) for k, v in data.items()}

In [6]:
matching_borders_counts = {}
for tile, tile_borders in borders.items():
    matching_borders_count = 0
    tmp_borders = borders.copy()
    tmp_borders.pop(tile)
    for border in tile_borders:
        matching_borders_count += any([np.array_equal(border, other_border) 
                                       for other_borders in tmp_borders.values() 
                                       for other_border in other_borders]) 
    matching_borders_counts[tile] = matching_borders_count

In [7]:
part1_ans = 1
for tile, count in matching_borders_counts.items():
    if count == 8:
        part1_ans *= tile
        
print(part1_ans)

111936085519519


#### Part 2